In [1]:
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

def cost_curve(x,premium,contribution,deductible,coinsuranceRate,maxOutOfPocket):
    eff_premium = premium - contribution
    post_deduct = (coinsuranceRate * (x - deductible)) + deductible
    y = np.fmin(np.fmin(x, post_deduct), maxOutOfPocket) + eff_premium
    return y

def plan_UI():
    plan = {
        "premium": Slider(start=0, end=2000, value=100, step=.1, title="Premium"),
        "contribution": Slider(start=0, end=1000, value=50, step=10, title="Company Contribution"),
        "deductible": Slider(start=0, end=6000, value=2000, step=10, title="Deductible"),
        "coinsurance": Slider(start=0, end=1, value=0.5, step=.01, title="Coinsurance Rate (paid by you)"),
        "maxOutOfPocket": Slider(start=0, end=8000, value=4000, step=10, title="Maximum Out Of Pocket")
    }
    return plan

callback_code = """
    const data = source.data;
    const P = premium.value - contribution.value;
    const D = deductible.value;
    const R = coinsurance.value;
    const M = maxOutOfPocket.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        const post_deduct = (R*(x[i] - D)) + D;
        y[i] = Math.min(Math.min(x[i],post_deduct),M) + P;
    }
    source.change.emit();
"""

In [2]:
x = np.linspace(0.0, 10000, 100)

plan1 = plan_UI()
plan2 = plan_UI()
y = cost_curve(x,plan1['premium'].value,plan1['contribution'].value,plan1['deductible'].value,plan1['coinsurance'].value,plan1['maxOutOfPocket'].value)

plot = figure(x_range=(0,10000), y_range=(0,10000), plot_width=400, plot_height=400, tools="pan,hover,crosshair,reset")
plot.line(x, x, line_dash=(4,4), line_color="black")
plot.xaxis.axis_label = 'Out-of-Pocket Expenses'
plot.yaxis.axis_label = 'Actual Incurred Cost'

source1 = ColumnDataSource(data=dict(x=x, y=y))
source2 = ColumnDataSource(data=dict(x=x, y=y))
plot.line('x', 'y', source=source1, line_width=3, line_alpha=0.6, legend="Plan 1")
plot.line('x', 'y', source=source2, line_width=3, line_alpha=0.6, legend="Plan 2", line_color="orange")

plan1.update(dict(source=source1))
plan2.update(dict(source=source2))

callback1 = CustomJS(args=plan1,code=callback_code)
callback2 = CustomJS(args=plan2,code=callback_code)

plan1['premium'].js_on_change('value', callback1)
plan1['contribution'].js_on_change('value', callback1)
plan1['deductible'].js_on_change('value', callback1)
plan1['coinsurance'].js_on_change('value', callback1)
plan1['maxOutOfPocket'].js_on_change('value', callback1)

plan2['premium'].js_on_change('value', callback2)
plan2['contribution'].js_on_change('value', callback2)
plan2['deductible'].js_on_change('value', callback2)
plan2['coinsurance'].js_on_change('value', callback2)
plan2['maxOutOfPocket'].js_on_change('value', callback2)

In [3]:
layout = row(
    plot,
    column(plan1['premium'],plan1['contribution'],plan1['deductible'],plan1['coinsurance'],plan1['maxOutOfPocket']),
    column(plan2['premium'],plan2['contribution'],plan2['deductible'],plan2['coinsurance'],plan2['maxOutOfPocket']),
)

output_file("main.html", title="Insurance Plan Comparison")

show(layout)